## Informações Gerais
 | Informações | Detalhes |
 |------------|-------------|
 |Nome Tabela | silver.clientes |
 |Origem | ADFS  |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |20/07/2025 | Tulio Augusto  | Criação do notebook |
 |20/07/2025 | Tulio Augusto  | Inclusão coluna valor |

In [0]:
%run /Workspace/Repos/AGRICOPEL/Agricopel-Databricks/FINANCEIRO/CONTAS_RECEBER/00.global/functions

In [0]:
from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
df_clientes = spark.read.parquet("abfss://bronze@storageagrucopel.dfs.core.windows.net/clientes")

In [0]:
database = "silver.financeiro"
tabela = "clientes"

In [0]:
df_clientes.createOrReplaceTempView("vw_dados")

In [0]:
df_clientes = spark.sql("""
    select
        distinct
        pais as nm_pais,
        tipo_cliente as nm_tipo_cliente,
        caminho_arquivo as nm_caminho_arquivo,
        current_timestamp() as dt_data_extracao
    from vw_dados
""")

In [0]:
df_clientes = df_clientes.withColumn("data_carga", current_date())
df_clientes = df_clientes.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))

In [0]:
comentario_tabela = 'Esta tabela é uma entidade corporativa que contém a relação dos clientes'

lista_comentario_colunas = {
'nm_pais' : 'Nome do país.',
'nm_tipo_cliente' :  'Tipo de cliente.',
'nm_caminho_arquivo' : 'Caminho do arquivo.',
'dt_data_extracao' : 'data extração bronze.',
'data_carga' : 'Data que o registro foi carregado',
'data_hora_carga' : 'Data e hora que o registro foi carregado'
}

In [0]:
df_clientes.write \
          .format('delta') \
          .mode('overwrite') \
          .option('overwriteSchema', 'true') \
          .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database, tabela, comentario_tabela, lista_comentario_colunas)
print("Dados gravados com sucesso!")

In [0]:
spark.sql(f"OPTIMIZE {database}.{tabela}")
print(f"Processo de otimização finalizado!.")